# QC a Processed Collection

In [ ]:
%load_ext autoreload
%autoreload 2

from src import *
from src.setup import *
from src.process import *
from src.qc import *

### `parameters`

In [ ]:
collection_name = "" # Define Collection name - example: "ble_12100302_Medina"
# collection_name = "Baxter_0_10_0"

### Instantiate CollectionData, Database, and JobClient objects

In [ ]:
collection = CollectionData(collection_name)
database = Database(collection)
job_client = JobClient(collection)

### Create Excel Error Report

In [ ]:
dfs = []
for step_name in collection.config["processing_steps"].keys():
    domain = collection.config["processing_steps"][step_name]["domain"]
    # Lets not capture the final status to preserve the timedout status jobs
    # job_client.poll_and_update_job_status(database, step_name, "models" if domain == "model" else "processing")
    _, _, failed_reaches = database.get_reach_status_by_process(
        step_name, "models" if domain == "model" else "processing"
    )
    df = job_client.get_failed_jobs_df(failed_reaches)
    dfs.append(df)
    write_failed_jobs_df_to_excel(df, step_name, collection.error_report_path)

### Create QC Map

In [ ]:
copy_qc_map(collection)

### Create Composite Rasters (flows2fim)

In [ ]:
run_flows2fim(collection)